In [1]:
import tensorflow as tf
print(tf.__version__)

2.9.0


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import img_to_array,load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import gc

In [ ]:
physical_devices=tf.config.experimental.list_physical_devices('GPU')
print("no of gpus",len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
if tf.test.gpu_device_name():
    print('default gpu device:{}'.format(tf.test.gpu_device_name()))
else:
    print("no gpu")


In [ ]:
dataset_path = "cell_images"

# Initialize lists to store images and labels
x = []  # List to store images
y = []  # List to store corresponding labels

# Define label mapping
label_map = {"Parasitized": 0, "Uninfected": 1}

# Iterate through each class folder
for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)

    # Iterate through each image in the class folder
    for image_filename in os.listdir(class_path):
        image_path = os.path.join(class_path, image_filename)

        # Read and preprocess the image (you may need to adjust the preprocessing)
        image = cv2.imread(image_path)
        # Add additional preprocessing steps if needed
        # ...

        # Append the preprocessed image to the x list
        x.append(image)

        # Assign the corresponding label based on the class folder
        label = label_map[class_folder]
        y.append(label)


In [ ]:
path="cell_images"
print(os.listdir(path))
classes=os.listdir(path)
print(len(classes))
print(classes)

In [ ]:
x = []

for class_folder in classes:
    class_path = os.path.join(path, class_folder)
    if os.path.isdir(class_path):
        class_images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]
        x.extend(class_images)

print(len(x))

In [ ]:
combined=list(zip(x,y))
print(combined[:10],"\n")
random.shuffle(combined)

print(combined[:10],"\n")
x[:],y[:] = zip(*combined)

In [ ]:
%matplotlib inline

plt.figure(figsize=(16,16))
for counter, i in enumerate(random.sample(range(0,len(x)) ,9)):
   plt.subplot(3,3,counter+1)
   plt.subplots_adjust(hspace=0.3)
   filename=x[i]
   image=imread(filename)
   plt.imshow(image)
   plt.title(y[i],fontsize=12)
plt.show() 

In [ ]:
class_num=os.listdir(dataset_path)
print(len(class_num))

In [ ]:
le=LabelEncoder()
le.fit(y)
y_ohe=to_categorical(le.transform(y),len(class_num))
print(y_ohe.shape)
y_ohe=np.array(y_ohe)

In [ ]:
img_data=np.array([img_to_array(load_img(img,target_size=(70,70))) for img in x])
print(img_data.shape)

x_train,x_test,y_train,y_test=train_test_split(img_data,y_ohe,test_size=0.2,random_state=2)

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=2)

print('Training Dataset Size',x_train.shape);
print('Validation Dataset Size',x_val.shape);
print('Test Dataset Size',x_test.shape);

In [ ]:
batch_size=32
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow(x_train,y_train,shuffle=False,batch_size=batch_size,seed=1)

val_datagen=ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow(x_val,y_val,
                                shuffle=False,
                                batch_size=batch_size,seed=1)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(70, 70, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.18),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.18),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.18),

    Dense(128,activation='relu'),
    BatchNormalization(),
    Dropout(0.18),
    Dense(2,activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#gc.collect()

In [ ]:
train_steps_per_epoch=x_train.shape[0]//batch_size
val_steps_per_epoch=x_val.shape[0]//batch_size
history=model.fit(train_generator,steps_per_epoch=train_steps_per_epoch,validation_data=val_generator,validation_steps=val_steps_per_epoch,epochs=30,verbose=1)

In [ ]:
x_test1=x_test/255
test_predictions=model.predict(x_test1)
predictions=le.classes_[np.argmax(test_predictions,axis=1)]
target_labels=le.classes_[np.argmax(y_test,axis=1)]
predict_df=pd.DataFrame({'Target_labels':target_labels,'Predictions':predictions})
predict_df.head(30)

In [ ]:
correct=(target_labels==predictions)
accuracy=correct.sum()/correct.size
print(accuracy)

from sklearn.metrics import precision_recall_fscore_support

# y_true is the true labels, y_pred is the predicted labels
# both y_true and y_pred should be arrays of integers representing the class labels
precision, recall, f1, support = precision_recall_fscore_support(target_labels, predictions, average='macro')

print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))

In [ ]:
cm = confusion_matrix(target_labels, predictions)

# Display the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, square=True)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [1]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.
